In [1]:
import pandas as pd
import scipy.special as sspecial
import numpy as np

ref_gebiet = "Bundesgebiet"

In [2]:
raw_data = pd.read_csv('https://www.bundeswahlleiterin.de/europawahlen/2024/ergebnisse/opendata/ew24/daten/kerg_00364.csv', skiprows=2, delimiter=";")

europe_election_df = raw_data\
.drop(raw_data.loc[:, raw_data.columns.str.startswith('Unnamed')], axis=1)\
.dropna(subset=['Nr'])\
.drop('Nr', axis=1)\
.fillna(0)\
.set_index('Gebiet', )\
.apply(pd.to_numeric)

europe_election_df.index = europe_election_df.index.astype('string')

/tmp/ipykernel_9726/3825362537.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(0)\


In [3]:
europe_election_percentage_df = pd.concat(
    [
        europe_election_df.iloc[:, 5] + europe_election_df.iloc[:, 9],
        europe_election_df.iloc[:, 6:]
    ], axis=1
)\
.drop('Christlich-Soziale Union in Bayern e.V.', axis=1)\
.div(europe_election_df['Gültige Stimmen'], axis=0)\
.rename({0: "CDU/CSU"}, axis=1)

In [4]:
ref_row = europe_election_percentage_df.loc[ref_gebiet]

In [5]:
rel_entr_df = pd.DataFrame(europe_election_percentage_df.apply(
    lambda x: sspecial.rel_entr(
        x.to_numpy(dtype=float),
        ref_row.to_numpy(dtype=float)
    ).sum(), axis=1
))

In [6]:
kl_div_df = pd.DataFrame(europe_election_percentage_df.apply(
    lambda x: sspecial.kl_div(
        x.to_numpy(dtype=float),
        ref_row.to_numpy(dtype=float)
    ).sum(), axis=1
))

In [7]:
europe_election_data_centered_df = europe_election_percentage_df.subtract(ref_row)
europe_election_centered_normalized_df = europe_election_data_centered_df.div(ref_row)

dist_normalized_df = pd.DataFrame(np.sqrt( np.square(europe_election_centered_normalized_df).sum(axis=1)))
dist_df = pd.DataFrame(np.sqrt( np.square(europe_election_data_centered_df).sum(axis=1)))

In [8]:
def ref_min_max(df, n=3):

    smalest = df.nsmallest(n=n+1, columns=0)
    largest = df.nlargest(n=n, columns=0)
    return pd.concat(
        [
            pd.DataFrame(pd.concat(
                [
                    smalest,
                    europe_election_df.loc[smalest.index],
                ],
                axis=1)
            ),
            pd.DataFrame(
                pd.concat(
                    [
                        largest,
                        europe_election_df.loc[largest.index],
                    ], 
                axis=1
                )
            )
        ]
    )

In [9]:
ref_min_max(rel_entr_df)

,0,gehört zu,Wahlberechtigte,Wählende,Ungültige Stimmen,Gültige Stimmen,Christlich Demokratische Union Deutschlands,BÜNDNIS 90/DIE GRÜNEN,Sozialdemokratische Partei Deutschlands,Alternative für Deutschland,...,Basisdemokratische Partei Deutschland,BÜNDNIS DEUTSCHLAND,Bündnis Sahra Wagenknecht - Vernunft und Gerechtigkeit,Demokratische Allianz für Vielfalt und Aufbruch,Klimaliste Deutschland,Parlament aufmischen – Stimme der Letzten Generation,Partei der Vernunft,Partei des Fortschritts,"V-Partei³ - Partei für Veränderung, Vegetarier und Veganer",Übrige
Gebiet,,,,,,,,,,,,,,,,,,,,,
Bundesgebiet,0.000000,0.0,61941751,40128348,332136,39796212,9431567,4736913,5548528,6324008,...,99502,164477,2453652,148724,31504,104340,29508,227631,55440,0
Wetteraukreis,0.010511,6.0,230160,148420,1262,147158,46520,17943,22629,21343,...,369,647,6251,499,117,299,99,872,272,0
Main-Kinzig-Kreis,0.010512,6.0,301947,185604,1636,183968,57373,19027,29277,30930,...,397,755,8702,1267,132,298,135,1152,324,0
Rhein-Neckar-Kreis,0.011167,8.0,404271,272630,3780,268850,82259,36361,38278,39751,...,578,1077,11920,983,211,523,186,1259,360,0
Eifelkreis Bitburg-Prüm,0.361862,7.0,72898,50891,665,50226,15864,2694,6622,5326,...,97,140,1721,33,19,53,41,183,59,0
Hildburghausen,0.340972,16.0,50316,33813,298,33515,6558,590,1910,12195,...,93,177,5219,13,20,29,29,80,18,0
Sonneberg,0.321870,16.0,45822,26089,171,25918,5534,528,1727,9952,...,40,144,4149,9,14,44,18,71,14,0


In [10]:
ref_min_max(kl_div_df)

,0,gehört zu,Wahlberechtigte,Wählende,Ungültige Stimmen,Gültige Stimmen,Christlich Demokratische Union Deutschlands,BÜNDNIS 90/DIE GRÜNEN,Sozialdemokratische Partei Deutschlands,Alternative für Deutschland,...,Basisdemokratische Partei Deutschland,BÜNDNIS DEUTSCHLAND,Bündnis Sahra Wagenknecht - Vernunft und Gerechtigkeit,Demokratische Allianz für Vielfalt und Aufbruch,Klimaliste Deutschland,Parlament aufmischen – Stimme der Letzten Generation,Partei der Vernunft,Partei des Fortschritts,"V-Partei³ - Partei für Veränderung, Vegetarier und Veganer",Übrige
Gebiet,,,,,,,,,,,,,,,,,,,,,
Bundesgebiet,0.000000,0.0,61941751,40128348,332136,39796212,9431567,4736913,5548528,6324008,...,99502,164477,2453652,148724,31504,104340,29508,227631,55440,0
Wetteraukreis,0.010511,6.0,230160,148420,1262,147158,46520,17943,22629,21343,...,369,647,6251,499,117,299,99,872,272,0
Main-Kinzig-Kreis,0.010512,6.0,301947,185604,1636,183968,57373,19027,29277,30930,...,397,755,8702,1267,132,298,135,1152,324,0
Rhein-Neckar-Kreis,0.011167,8.0,404271,272630,3780,268850,82259,36361,38278,39751,...,578,1077,11920,983,211,523,186,1259,360,0
Eifelkreis Bitburg-Prüm,0.361862,7.0,72898,50891,665,50226,15864,2694,6622,5326,...,97,140,1721,33,19,53,41,183,59,0
Hildburghausen,0.340972,16.0,50316,33813,298,33515,6558,590,1910,12195,...,93,177,5219,13,20,29,29,80,18,0
Sonneberg,0.321870,16.0,45822,26089,171,25918,5534,528,1727,9952,...,40,144,4149,9,14,44,18,71,14,0


In [11]:
ref_min_max(dist_normalized_df)

,0,gehört zu,Wahlberechtigte,Wählende,Ungültige Stimmen,Gültige Stimmen,Christlich Demokratische Union Deutschlands,BÜNDNIS 90/DIE GRÜNEN,Sozialdemokratische Partei Deutschlands,Alternative für Deutschland,...,Basisdemokratische Partei Deutschland,BÜNDNIS DEUTSCHLAND,Bündnis Sahra Wagenknecht - Vernunft und Gerechtigkeit,Demokratische Allianz für Vielfalt und Aufbruch,Klimaliste Deutschland,Parlament aufmischen – Stimme der Letzten Generation,Partei der Vernunft,Partei des Fortschritts,"V-Partei³ - Partei für Veränderung, Vegetarier und Veganer",Übrige
Gebiet,,,,,,,,,,,,,,,,,,,,,
Bundesgebiet,0.000000,0.0,61941751,40128348,332136,39796212,9431567,4736913,5548528,6324008,...,99502,164477,2453652,148724,31504,104340,29508,227631,55440,0
Rhein-Neckar-Kreis,1.243447,8.0,404271,272630,3780,268850,82259,36361,38278,39751,...,578,1077,11920,983,211,523,186,1259,360,0
Niedersachsen,1.317000,99.0,6191243,3969241,19704,3949537,1239841,480893,770237,522920,...,8207,16366,176127,12680,2579,9485,2767,26063,4968,0
Wetteraukreis,1.389723,6.0,230160,148420,1262,147158,46520,17943,22629,21343,...,369,647,6251,499,117,299,99,872,272,0
"Gelsenkirchen, Stadt",9.763457,5.0,169647,88700,796,87904,20630,6768,18942,19043,...,91,344,4296,2300,34,147,43,539,115,0
Eifelkreis Bitburg-Prüm,8.186936,7.0,72898,50891,665,50226,15864,2694,6622,5326,...,97,140,1721,33,19,53,41,183,59,0
"Offenbach am Main, Stadt",8.067288,6.0,68949,36478,245,36233,8170,5595,5749,4044,...,89,90,2100,542,46,166,18,164,73,0


In [12]:
ref_min_max(dist_df)

,0,gehört zu,Wahlberechtigte,Wählende,Ungültige Stimmen,Gültige Stimmen,Christlich Demokratische Union Deutschlands,BÜNDNIS 90/DIE GRÜNEN,Sozialdemokratische Partei Deutschlands,Alternative für Deutschland,...,Basisdemokratische Partei Deutschland,BÜNDNIS DEUTSCHLAND,Bündnis Sahra Wagenknecht - Vernunft und Gerechtigkeit,Demokratische Allianz für Vielfalt und Aufbruch,Klimaliste Deutschland,Parlament aufmischen – Stimme der Letzten Generation,Partei der Vernunft,Partei des Fortschritts,"V-Partei³ - Partei für Veränderung, Vegetarier und Veganer",Übrige
Gebiet,,,,,,,,,,,,,,,,,,,,,
Bundesgebiet,0.000000,0.0,61941751,40128348,332136,39796212,9431567,4736913,5548528,6324008,...,99502,164477,2453652,148724,31504,104340,29508,227631,55440,0
Rhein-Neckar-Kreis,0.033442,8.0,404271,272630,3780,268850,82259,36361,38278,39751,...,578,1077,11920,983,211,523,186,1259,360,0
Main-Kinzig-Kreis,0.035331,6.0,301947,185604,1636,183968,57373,19027,29277,30930,...,397,755,8702,1267,132,298,135,1152,324,0
Wetteraukreis,0.036305,6.0,230160,148420,1262,147158,46520,17943,22629,21343,...,369,647,6251,499,117,299,99,872,272,0
Sonneberg,0.292356,16.0,45822,26089,171,25918,5534,528,1727,9952,...,40,144,4149,9,14,44,18,71,14,0
Görlitz,0.292323,14.0,202526,137929,2125,135804,32188,3809,5945,54431,...,391,734,16731,41,50,215,190,493,137,0
Spree-Neiße,0.291924,12.0,94005,64604,1290,63314,10894,1348,6712,23961,...,123,349,9807,23,31,102,69,263,58,0
